In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

In [2]:
%%capture
!pip install libauc==1.2.0
!pip install medmnist
!pip install torchio

In [3]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

from libauc.models import resnet18
from libauc.sampler import DualSampler
from libauc.metrics import auc_prc_score

import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter
from torchio import Image

In [4]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.2.1 @ https://github.com/MedMNIST/MedMNIST/


**BREASTMNIST DATA**

In [5]:
data_flag = 'breastmnist'
# data_flag = 'pneumoniamnist'
#data_flag = 'chestmnist'

download = True

#NUM_EPOCHS = 3
BATCH_SIZE = 128
#lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [6]:
print(info)
print(n_channels)
print(n_classes)
print(DataClass)

{'python_class': 'BreastMNIST', 'description': 'The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.', 'url': 'https://zenodo.org/record/6496656/files/breastmnist.npz?download=1', 'MD5': '750601b1f35ba3300ea97c75c52ff8f6', 'task': 'binary-class', 'label': {'0': 'malignant', '1': 'normal, benign'}, 'n_channels': 1, 'n_samples': {'train': 546, 'val': 78, 'test': 156}, 'license': 'CC BY 4.0'}
1
2
<class 'medmnist.dataset.BreastMNIST'>


In [7]:
from torchio.transforms import RandomAffine, RandomFlip, RandomNoise, RandomGamma
class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=28, crop_size=24, mode='train', kernel_size=3):
        self.images = images.astype(np.uint8)
        self.targets = targets
        self.mode = mode

        
        self.transform_train = transforms.Compose([
                                                    transforms.ToTensor(),
                                                    transforms.Resize((image_size, image_size)),
                                                    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.2), shear=0.2),
                                                    transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)], p=0.5),
                                                    transforms.RandomHorizontalFlip(p=0.5),
                                                    transforms.RandomVerticalFlip(p=0.5),
                                                    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
                                                ])

        self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             #transforms.GaussianBlur(kernel_size=(kernel_size, kernel_size), sigma=(0.1, 2.0)),
                             #transforms.BilateralFilter(diameter=5, sigma_color=0.1, sigma_space=15),
                             transforms.Resize((image_size, image_size)),
                              ])
        
        
        # for loss function
        self.pos_indices = np.flatnonzero(targets==1)
        self.pos_index_map = {}
        for i, idx in enumerate(self.pos_indices):
            self.pos_index_map[idx] = i

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        #image = torch.from_numpy(image).unsqueeze(0) 
        if self.mode == 'train':
           idx = self.pos_index_map[idx] if idx in self.pos_indices else -1
           #image = image.filter(ImageFilter.GaussianBlur(radius=3))
           #image = image.filter(ImageFilter.BLUR)
           #image = image.filter(ImageFilter.UnsharpMask(radius=3, percent=150, threshold=3))

           image = self.transform_train(image)
        else:
           #image = image.filter(ImageFilter.GaussianBlur(radius=3))
           #image = image.filter(ImageFilter.BLUR)
           #image = image.filter(ImageFilter.UnsharpMask(radius=3, percent=150, threshold=3))
           image = self.transform_test(image)
        return idx, image, target 

In [8]:
train_dataset = DataClass(split='train', download=download)
print(train_dataset[0])
test_dataset = DataClass(split='test', download=download)
print(test_dataset[0])

100%|██████████| 559580/559580 [00:01<00:00, 515755.52it/s]

(<PIL.Image.Image image mode=L size=28x28 at 0x7668FE562F20>, array([1]))
Using downloaded and verified file: /root/.medmnist/breastmnist.npz
(<PIL.Image.Image image mode=L size=28x28 at 0x7668FE562F20>, array([0]))


In [9]:
train_images = np.array([np.asarray(image) for (image, target) in train_dataset])
train_labels = np.array([target for (image, target) in train_dataset])
print(type(train_images[0]))

<class 'numpy.ndarray'>


In [10]:
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
print(type(test_images[0]))

<class 'numpy.ndarray'>


In [11]:
test_images[0].shape

(28, 28)

In [12]:
batch_size = 32
sampling_rate = 0.5

trainSet = ImageDataset(train_images, train_labels)
trainSet_eval = ImageDataset(train_images, train_labels, mode='test')
testSet = ImageDataset(test_images, test_labels, mode='test')

sampler = DualSampler(trainSet, batch_size, sampling_rate=sampling_rate)
train_loader = torch.utils.data.DataLoader(trainSet, batch_size=batch_size, sampler=sampler, num_workers=2)
train_loader_at_eval = torch.utils.data.DataLoader(trainSet_eval, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=False, num_workers=2)

In [13]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset BreastMNIST (breastmnist)
    Number of datapoints: 546
    Root location: /root/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'malignant', '1': 'normal, benign'}
    Number of samples: {'train': 546, 'val': 78, 'test': 156}
    Description: The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
    License: CC BY 4.0
Dataset BreastMNIST (breastmnist)
    Number of datapoints: 156
    Root location: /root/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'malignant', '1'

In [ ]:
#print(train_dataset[0][0].shape)

# # montage
train_dataset.montage(length=20)

**Checking Imbalance in Dataset**

In [20]:
from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
# from libauc.models import densenet121 as DenseNet121


from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torchvision.models as models
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [21]:
#configure GPU
device = torch.device(0 if torch.cuda.is_available() else 'cpu')

In [22]:
import warnings
warnings.filterwarnings("ignore")

Pretraining

In [23]:
import os, random
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    #torch.cuda.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #os.environ["PYTHONHASHSEED"]=str(SEED)
    #random.seed(SEED)
    #torch.use_deterministic_algorithms(True)

In [62]:
# paramaters
SEED = 123
BATCH_SIZE = 32
lr = 1e-4
weight_decay = 1e-5
num_classes=1

# model
model = models.resnet18(pretrained=True)
# model = models.resnet18(pretrained=False)

input_channel = model.fc.in_features
#for param in model.parameters():
    #param.requires_grad = False
#model.fc = nn.Sequential(nn.Linear(input_channel, 128),
 #                        nn.ReLU(),
  #                       nn.Linear(128, num_classes),
   #                      nn.Sigmoid()
    #                     )
model.fc = nn.Sequential(nn.Linear(input_channel, 128),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.2),
                                 nn.Linear(128, 32),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.1),
                                 nn.Linear(32, num_classes),
                                 nn.Sigmoid()
                                 )


model = model.to(device)

# create a binary cross-entropy loss function
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, patience=3,  verbose=True, factor=0.5, 
                              threshold=0.001, min_lr=0.00001, mode = 'max')

# training
best_val_auc = 0 
for epoch in range(30):
    for idx, (index, data, labels) in enumerate(train_loader):
      train_data, train_labels = data.to(device), labels.to(device)
      train_data=train_data.repeat(1,3,1,1)
#       train_labels=train_labels.float()

#       train_data, train_labels  = train_data.to(device), train_labels.to(device)
      # break
      #train_data = torch.tensor(train_data,requires_grad=False)
      y_pred = model(train_data)
      
      #y_pred=torch.where(y_pred > 0.5, torch.tensor([1.], device=device), torch.tensor([0.], device=device))
      #y_pred=torch.where(y_pred > 0.5, torch.tensor([1.], device=device), torch.tensor([0.], device=device))

      #print(y_pred)
      #y_pred = [1 if x > 0.5 else 0 for x in y_pred]
      loss = criterion(y_pred, train_labels.float())
      #print("Training Loss= ", loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
    print(f'Validating epoch: {epoch+1} Loss: {loss.item()}')  
      # validation  
      #if idx % 100 == 0:
      #if (1):
    model.eval()
    val_auc_mean = 0
    with torch.no_grad():    
         val_pred = []
         val_true = [] 
         for jdx, (index, data, targets) in enumerate(train_loader_at_eval):
             val_data, val_labels = data.to(device), targets.to(device)
             val_data=val_data.repeat(1,3,1,1)
#              val_labels=val_labels.float()
#              val_data = val_data.to(device)
             y_pred = model(val_data)
             val_pred.append(y_pred.cpu().detach().numpy())
             val_true.append(val_labels.cpu().detach().numpy())
     
         val_true = np.concatenate(val_true)
         val_pred = np.concatenate(val_pred)
         #val_pred = [1 if x > 0.5 else 0 for x in val_pred]
         val_auc_mean =  roc_auc_score(val_true, val_pred) 
         #model.train
         if best_val_auc < val_auc_mean:
            best_val_auc = val_auc_mean
            torch.save(model.state_dict(), 'ce_bmnist_pretrained_model.pth')
         print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, val_auc_mean, best_val_auc) )
    scheduler.step(val_auc_mean)

Validating epoch: 1 Loss: 0.6509397029876709
Epoch=0, BatchID=23, Val_AUC=0.6057, Best_Val_AUC=0.6057
Validating epoch: 2 Loss: 0.675871729850769
Epoch=1, BatchID=23, Val_AUC=0.7939, Best_Val_AUC=0.7939
Validating epoch: 3 Loss: 0.5604209303855896
Epoch=2, BatchID=23, Val_AUC=0.8350, Best_Val_AUC=0.8350
Validating epoch: 4 Loss: 0.4964093565940857
Epoch=3, BatchID=23, Val_AUC=0.8480, Best_Val_AUC=0.8480
Validating epoch: 5 Loss: 0.4227776527404785
Epoch=4, BatchID=23, Val_AUC=0.8707, Best_Val_AUC=0.8707
Validating epoch: 6 Loss: 0.7316681146621704
Epoch=5, BatchID=23, Val_AUC=0.8415, Best_Val_AUC=0.8707
Validating epoch: 7 Loss: 0.5533223748207092
Epoch=6, BatchID=23, Val_AUC=0.8647, Best_Val_AUC=0.8707
Validating epoch: 8 Loss: 0.4867185354232788
Epoch=7, BatchID=23, Val_AUC=0.8812, Best_Val_AUC=0.8812
Validating epoch: 9 Loss: 0.3608716130256653
Epoch=8, BatchID=23, Val_AUC=0.8828, Best_Val_AUC=0.8828
Validating epoch: 10 Loss: 0.4762222468852997
Epoch=9, BatchID=23, Val_AUC=0.9087, 

In [63]:
    print(f'Testing ...')  
    PATH = 'ce_bmnist_pretrained_model.pth' 
    model_state_dict = torch.load(PATH)
    #model = MyModel()  # Create an instance of your model
    model.load_state_dict(model_state_dict)  # Load the saved parameters into the model
    with torch.no_grad():    
         test_pred = []
         test_true = [] 
         for jdx, (index, data, targets) in enumerate(test_loader):
             test_data, test_labels = data.to(device), targets.to(device)
             test_data=test_data.repeat(1,3,1,1)
#              test_labels=test_labels.float()
#              test_data = test_data.to(device)
             y_pred = model(test_data)
             test_pred.append(y_pred.cpu().detach().numpy())
             test_true.append(test_labels.cpu().detach().numpy())
     
         test_true = np.concatenate(test_true)
         test_pred = np.concatenate(test_pred)
         #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
         test_auc_mean =  roc_auc_score(test_true, test_pred) 
        # model.train
        # if best_val_auc < val_auc_mean:
        #    best_val_auc = val_auc_mean
        #    #torch.save(model.state_dict(), 'ce_pretrained_model.pth')
         print('Test_AUC=%.4f'%( test_auc_mean))

Testing ...
Test_AUC=0.9252


AUCM Optimization

In [68]:
from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *
from torch.optim.lr_scheduler import ExponentialLR


# paramaters
SEED = 123
#imratio = train_dataset.imratio
lr =0.01#0.05# 0.04#0.05 # using smaller learning rate is better [0.001, 0.01, 0.05, 0.1]
epoch_decay = 2e-3#1e-4
weight_decay = 1e-5#1e-5
margin = 1.0

# model
set_all_seeds(SEED)
# model
model = models.resnet18(pretrained=False)
input_channel = model.fc.in_features
#for param in model.parameters():
    #param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(input_channel, 128),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.2),
                                 nn.Linear(128, 32),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.1),
                                 nn.Linear(32, num_classes),
                                 nn.Sigmoid()
                                 )


model = model.to(device)

# load pretrained model
if True:
  PATH = 'ce_bmnist_pretrained_model.pth' 
  state_dict = torch.load(PATH)
  state_dict.pop('classifier.weight', None)
  state_dict.pop('classifier.bias', None) 
  model.load_state_dict(state_dict, strict=False)


# define loss & optimizer
loss_fn = AUCMLoss()
optimizer = PESG(model, 
                 loss_fn=loss_fn, 
                 lr=lr, 
                 margin=margin, 
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
#lr_scheduler_opt = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
#optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, patience=2,  verbose=True, factor=0.5, 
                              threshold=0.001, min_lr=0.00001, mode = 'max')
#torch_lr_scheduler = ExponentialLR(optimizer=optimizer, gamma=0.98)
#scheduler = create_lr_scheduler_with_warmup(torch_lr_scheduler,
 #                                           warmup_start_value=0.05,
  #                                          warmup_end_value=0.1,
   #                                         warmup_duration=2)

best_val_auc = 0
for epoch in range(30):
  if epoch%10 == 0 and epoch > 0:
      optimizer.update_regularizer(decay_factor=10)
  #lr_scheduler_opt.step()
  
  for idx, (index, data, labels) in enumerate(train_loader):
      train_data, train_labels = data.to(device), labels.to(device)
      train_data=train_data.repeat(1,3,1,1)
#       train_labels=train_labels.float()

#       train_data, train_labels = train_data.to(device), train_labels.to(device)
      y_pred = model(train_data)
      #y_pred = torch.sigmoid(y_pred)
      #y_pred=torch.where(y_pred > 0.5, torch.tensor([1.], device=device), torch.tensor([0.], device=device))
      #y_pred = [1 if x > 0.5 else 0 for x in y_pred]
      #print(y_pred)
      loss = loss_fn(y_pred, train_labels.float())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # validation
  #if idx % 400 == 0:
  print(f'Validating epoch: {epoch+1} Loss: {loss.item()}')
  model.eval()
  val_auc = 0
  with torch.no_grad():    
        test_pred = []
        test_true = [] 
        for jdx, (index, data, targets) in enumerate(train_loader_at_eval):
            test_data, test_label = data.to(device), targets.to(device)
            test_data=test_data.repeat(1,3,1,1)
#             test_labels=test_labels.float()
#             test_data = test_data.to(device)
            y_pred = model(test_data)
            test_pred.append(y_pred.cpu().detach().numpy())
            test_true.append(test_label.cpu().detach().numpy())
        
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
        val_auc =  roc_auc_score(test_true, test_pred) 
        #model.train(
        if best_val_auc < val_auc:
           best_val_auc = val_auc
        #if epoch==99:
           torch.save(model, 'aucm_trained_model_breastmnist.pth')
  scheduler.step(val_auc)
  lr_scheduler.step()
  #scheduler.step()
  print ('Epoch=%s, BatchID=%s, Val_AUC=%.7f, lr=%.15f'%(epoch, idx, val_auc,  optimizer.lr))
  #for param_group in optimizer.param_groups:
   #     print('Epoch {}, LR: {}'.format(epoch, param_group['lr']))
print ('Best Val_AUC is %.4f'%best_val_auc)

        

Validating epoch: 1 Loss: 0.1740417182445526
Epoch=0, BatchID=23, Val_AUC=0.4805892, lr=0.010000000000000
Validating epoch: 2 Loss: 0.18998673558235168
Epoch=1, BatchID=23, Val_AUC=0.9289380, lr=0.010000000000000
Validating epoch: 3 Loss: 0.1676190197467804
Epoch=2, BatchID=23, Val_AUC=0.9327741, lr=0.010000000000000
Validating epoch: 4 Loss: 0.14093293249607086
Epoch=3, BatchID=23, Val_AUC=0.9480845, lr=0.010000000000000
Validating epoch: 5 Loss: 0.1261535882949829
Epoch=4, BatchID=23, Val_AUC=0.9445553, lr=0.010000000000000
Validating epoch: 6 Loss: 0.09620046615600586
Epoch=5, BatchID=23, Val_AUC=0.9439415, lr=0.010000000000000
Validating epoch: 7 Loss: 0.11482340097427368
Epoch=6, BatchID=23, Val_AUC=0.9605817, lr=0.010000000000000
Validating epoch: 8 Loss: 0.04605229198932648
Epoch=7, BatchID=23, Val_AUC=0.9548361, lr=0.010000000000000
Validating epoch: 9 Loss: 0.11416162550449371
Epoch=8, BatchID=23, Val_AUC=0.9504544, lr=0.010000000000000
Validating epoch: 10 Loss: 0.07152192294

In [69]:
print(f'Testing ...')  
PATH = 'aucm_trained_model_breastmnist.pth' 
model = torch.load(PATH)
#model = MyModel()  # Create an instance of your model
##model=torch.load(model_dumped)  # Load the saved parameters into the model
best_val_auc=0.0
with torch.no_grad():    
     test_pred = []
     test_true = [] 
     for jdx, (index, data, targets) in enumerate(test_loader):
         test_data, test_labels = data.to(device), targets.to(device)
         test_data=test_data.repeat(1,3,1,1)
#          test_labels=test_labels.float()
#          test_data = test_data.to(device)
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_labels.cpu().detach().numpy())
 
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
     #print(test_pred)
     #print(test_true)
     val_auc_mean =  roc_auc_score((test_true), test_pred) 
    # model.train
    # if best_val_auc < val_auc_mean:
    #    best_val_auc = val_auc_mean
    #    #torch.save(model.state_dict(), 'ce_pretrained_model.pth')
     print('Test_AUC=%.7f'%( val_auc_mean))

Testing ...
Test_AUC=0.9252297


Reproducibility Check

In [28]:
 PATH = '/kaggle/input/model-bm/aucm_trained_model_breastmnist.pth'
print(f'Testing ...')  
#PATH = 'aucm_trained_model_breastmnist_gaussian.pth' 
 model = torch.load(PATH)
#model = MyModel()  # Create an instance of your model
##model=torch.load(model_dumped)  # Load the saved parameters into the model
best_val_auc=0.0
with torch.no_grad():    
     test_pred = []
     test_true = [] 
     for jdx, (index, data, targets) in enumerate(test_loader):
         test_data, test_labels = data.to(device), targets.to(device)
         test_data=test_data.repeat(1,3,1,1)
#          test_labels=test_labels.float()
#          test_data = test_data.to(device)
         y_pred = model(test_data)
         test_pred.append(y_pred.cpu().detach().numpy())
         test_true.append(test_labels.cpu().detach().numpy())
 
     test_true = np.concatenate(test_true)
     test_pred = np.concatenate(test_pred)
     #test_pred = [1 if x > 0.5 else 0 for x in test_pred]
     #print(test_pred)
     #print(test_true)
     val_auc_mean =  roc_auc_score(test_true, test_pred) 
    # model.train
    # if best_val_auc < val_auc_mean:
    #    best_val_auc = val_auc_mean
    #    #torch.save(model.state_dict(), 'ce_pretrained_model.pth')
     print('Test_AUC=%.7f'%( val_auc_mean))

Testing ...
Test_AUC=0.9252297
